# Import

In [27]:
import csv
import os
import sys

import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

## Add configuration file

In [28]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [29]:
from ALL import config 
from util import *

## Set condition

In [30]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [31]:
data_type="AgNews"
vectorize_type = "sentenceBERT"

# Read data

In [32]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [33]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [34]:
max_vector_model_nums = config["vectorize"][vectorize_type]["max_model_num"]
vector_dims = config["vectorize"][vectorize_type]["dims"]

# Functions

In [35]:
def centralize_array(array):
    return array - np.mean(array, axis=0)

In [36]:
def normarize_array(array):
    return array / np.sqrt(np.sum(array * array, axis=1).reshape(-1, 1))

In [37]:
def normarize_vector(vector):
    return vector / np.sqrt(np.sum(vector * vector))

In [38]:
def get_average_vector(vectors):
    sum_vector = np.sum(vectors, axis=0)
    return normarize_vector(sum_vector)

# Centralize

In [39]:
vectors_path = f"../../Vectorize/data/{data_type}/{vectorize_type}/vector/umap"
converted_vectors_path = f"../data/{data_type}/{vectorize_type}/vector"
for vector_model_num in range(max_vector_model_nums):
    for vector_dim in tqdm(vector_dims):
        vector = np.load(
            f"{vectors_path}/{vector_dim}/{vector_model_num}.npy",
        )
        centralized_vector = centralize_array(vector)
        normarized_vector = normarize_array(centralized_vector)

        np.save(
            make_filepath(
                f"{converted_vectors_path}/{vector_dim}/centralized/{vector_model_num}.npy"
            ),
            centralized_vector,
        )
        np.save(
            make_filepath(
                f"{converted_vectors_path}/{vector_dim}/normalized/{vector_model_num}.npy"
            ),
            normarized_vector,
        )

100%|██████████| 9/9 [00:00<00:00, 581.96it/s]
